In [2]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'Fact_Customer_Orders'
 
sf_df = my_session.sql("select PRODUCT_ID,ORDER_QTY,ORDER_DATE from {}".format(table_name))
df = sf_df.to_pandas()

In [7]:
df.columns

Index(['PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [8]:
data = df[['PRODUCT_ID','ORDER_QTY','ORDER_DATE']]

In [9]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_QTY   225563 non-null  int16         
 2   ORDER_DATE  225563 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [12]:
data = data.set_index('ORDER_DATE')

In [13]:
data.sort_index(inplace=True)

In [14]:
data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2019-01-02,Product_001,850
2019-01-02,Product_004,700
2019-01-02,Product_004,900
2019-01-02,Product_004,550
2019-01-02,Product_004,300
...,...,...
2024-08-31,Product_098,200
2024-08-31,Product_100,800
2024-08-31,Product_100,550


In [15]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [16]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'test_forecast_order_qty','rse', 'rmse','r2_score'])

In [17]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'forecast_order_qty'])

In [18]:
for PRODUCT_ID, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    
    date_list = test_df.index.to_list()
    
    new_row = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list,
        'test_forecast_order_qty': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

In [19]:
test_results

,PRODUCT_ID,ORDER_DATE,test_forecast_order_qty,rse,rmse,r2_score
0,Product_001,13530,537.353131,0.0,290.185585,-0.000545
1,Product_001,13531,549.804686,0.0,290.185585,-0.000545
2,Product_001,13532,538.368357,0.0,290.185585,-0.000545
3,Product_001,13533,553.529134,0.0,290.185585,-0.000545
4,Product_001,13534,525.990260,0.0,290.185585,-0.000545
...,...,...,...,...,...,...
45125,Product_100,3886,476.050933,0.0,295.283780,-0.021838
45126,Product_100,3887,522.781958,0.0,295.283780,-0.021838
45127,Product_100,3888,475.125436,0.0,295.283780,-0.021838
45128,Product_100,3889,498.165467,0.0,295.283780,-0.021838
